In [16]:
from scipy.stats import ranksums
import pandas as pd

## Utils and Library for notebook
from notebook_utils.OpenKbcMSToolkit import ExtractionToolkit as exttoolkit

# Root data path
DATA_PATH = '../data/'

#Data loading
df = pd.read_csv(DATA_PATH+"second_level_data/msigdb_activation_vst_CD8.csv", engine='c', index_col=0).T.dropna()
meta_data = pd.read_csv(DATA_PATH+'annotation_metadata/EPIC_HCvB_metadata_baseline_updated-share.csv')
longDD_samples, shortDD_samples = exttoolkit.get_sample_name_by_contValues(meta_data, 'HCVB_ID', 'DiseaseDuration', 50)
longDD_samples = list(set(longDD_samples.values.tolist()).intersection(df.columns.tolist())) # intersected with act score matrix
shortDD_samples = list(set(shortDD_samples.values.tolist()).intersection(df.columns.tolist())) # intersected with act score matrix
#df = df[longDD_samples+shortDD_samples].dropna() # reform df with intersected samples

/Users/junheeyun/OpenKBC/multiple_sclerosis_proj/notebook/notebook_utils/OpenKbcMSToolkit.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cont_df[dataColname] = cont_df[dataColname].astype(float) # make float


In [17]:
significant_list = []
for x in df.index.tolist():
    long_data = df[longDD_samples].loc[x] # Long expr list
    short_data = df[shortDD_samples].loc[x] # Short expr list

    s, p = ranksums(long_data.values.tolist(), short_data.values.tolist()) # ranksum
    fc = short_data.mean(skipna=True) - long_data.mean(skipna=True) # FC

    if p<0.05:
        significant_list.append([x,fc, p]) # sig list

sig_df = pd.DataFrame(significant_list, columns=["Names", "fc", "pval"])

In [18]:
print(sig_df.sort_values(by=['fc'], ascending=False))
df.loc[sig_df["Names"].values.tolist()].to_csv("resultFiles/Ranksum.CD8.csv")

                                                  Names        fc      pval
2960  GSE5542_UNTREATED_VS_IFNG_TREATED_EPITHELIAL_C...  0.472934  0.021005
2961  GSE5542_UNTREATED_VS_IFNA_TREATED_EPITHELIAL_C...  0.462940  0.023127
2957  GSE5542_UNTREATED_VS_IFNG_TREATED_EPITHELIAL_C...  0.451128  0.025134
2956          GSE4984_LPS_VS_VEHICLE_CTRL_TREATED_DC_DN  0.438776  0.026039
2958  GSE5542_UNTREATED_VS_IFNA_TREATED_EPITHELIAL_C...  0.375618  0.020502
...                                                 ...       ...       ...
5813                                         MODULE_151 -0.355433  0.006635
5809                                         MODULE_114 -0.355556  0.006544
3568  FAN_OVARY_CL10_PUTATIVE_EARLY_ATRESIA_GRANULOS... -0.355558  0.002337
3465     BUSSLINGER_DUODENAL_DIFFERENTIATING_STEM_CELLS -0.362739  0.007714
3567      FAN_OVARY_CL8_MATURE_CUMULUS_GRANULOSA_CELL_2 -0.404299  0.002068

[6155 rows x 3 columns]
